## Loading Libraries

In [1]:
import dash
#from dash import dcc, html, Input, Output, State, callback_context
import dash_bootstrap_components as dbc
import folium
import pandas as pd
import plotly
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import Dash
from dash import dcc
from dash import html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
import pandas as pd
import json
import folium
from dash import dcc, html, Input, Output, State
import unicodedata

## Load Data

In [2]:
bancos_data = pd.read_excel('/Users/ikermontane/Documents/Github_Dash/Banamex_DASHBOARD_Proyect/FInal_Dashboard/Bancos.xlsx')
data = pd.read_excel('/Users/ikermontane/Documents/Github_dash/Banamex_DASHBOARD_Proyect/FInal_Dashboard/merge_clean.xlsx')
cluster_0 = pd.read_excel('/Users/ikermontane/Documents/Github_dash/Banamex_DASHBOARD_Proyect/FInal_Dashboard/cluster_0_SubCluster.xlsx')

### Load JSONs

In [3]:
def remove_accents_and_clean(value):
    if isinstance(value, str):
        # Normalize string to remove accents
        normalized = unicodedata.normalize('NFD', value)
        without_accents = ''.join(c for c in normalized if unicodedata.category(c) != 'Mn')
        # Capitalize the string
        return without_accents.title()
    return value


guanajuato = '/Users/ikermontane/Documents/Github_dash/Banamex_DASHBOARD_Proyect/FInal_Dashboard/Guanajuato.json'
with open(guanajuato, 'r') as f:
    guanajuato_geojson = json.load(f)

# Extract unique municipio names from the JSON data
unique_municipios = list(set(feature['properties']['NOMGEO'] for feature in guanajuato_geojson['features']))

# Clean and replace municipio names
cleaned_municipios = [remove_accents_and_clean(municipio) for municipio in unique_municipios]

# Replace the municipio names in the JSON data
for feature in guanajuato_geojson['features']:
    original = feature['properties']['NOMGEO']
    cleaned = remove_accents_and_clean(original)
    feature['properties']['NOMGEO'] = cleaned
    
bancos_data["municipio"] = bancos_data["municipio"].apply(remove_accents_and_clean)

mexico = '/Users/ikermontane/Documents/Github_dash/Banamex_DASHBOARD_Proyect/FInal_Dashboard/México.json'
with open(mexico, 'r') as f:
    mexico_geojson = json.load(f)

# Extract unique municipio names from the JSON data
unique_municipios = list(set(feature['properties']['NOMGEO'] for feature in mexico_geojson['features']))

# Clean municipio names and replace in the JSON data
for feature in mexico_geojson['features']:
    original = feature['properties']['NOMGEO']
    cleaned = remove_accents_and_clean(original)
    feature['properties']['NOMGEO'] = cleaned

# Extract cleaned and sorted unique municipio names
cleaned_municipios = sorted(set(feature['properties']['NOMGEO'] for feature in mexico_geojson['features']))

## Functions

### Creating a map

In [4]:
# Create the map with filtering functionality
def create_map(data, guanajuato_geojson, mexico_geojson, selected_cluster):
    def get_color(cluster):
        return "green" if cluster == selected_cluster else "gray"

    # Update GeoJSON properties
    for feature in guanajuato_geojson['features']:
        municipio_name = feature['properties']['NOMGEO']
        cluster_value = data.loc[data['Municipio'] == municipio_name, 'Cluster'].values
        if len(cluster_value) > 0:
            feature['properties']['Cluster'] = int(cluster_value[0])
        else:
            feature['properties']['Cluster'] = None

    for feature in mexico_geojson['features']:
        municipio_name = feature['properties']['NOMGEO']
        cluster_value = data.loc[data['Municipio'] == municipio_name, 'Cluster'].values
        if len(cluster_value) > 0:
            feature['properties']['Cluster'] = int(cluster_value[0])
        else:
            feature['properties']['Cluster'] = None

    m = folium.Map(location=[20.5, -100.5], zoom_start=7)

    # Add layers
    folium.GeoJson(
        guanajuato_geojson,
        name="Guanajuato",
        style_function=lambda x: {
            'fillColor': get_color(x['properties']['Cluster']),
            'color': 'black',
            'fillOpacity': 0.7,
        },
        tooltip=folium.GeoJsonTooltip(fields=["NOMGEO", "Cluster"], aliases=["Municipio:", "Cluster:"])
    ).add_to(m)

    folium.GeoJson(
        mexico_geojson,
        name="México",
        style_function=lambda x: {
            'fillColor': get_color(x['properties']['Cluster']),
            'color': 'black',
            'fillOpacity': 0.7,
        },
        tooltip=folium.GeoJsonTooltip(fields=["NOMGEO", "Cluster"], aliases=["Municipio:", "Cluster:"])
    ).add_to(m)

    folium.LayerControl().add_to(m)

    return m._repr_html_()

### Creating elements with-in a dashboard

In [5]:
# Function to create the dropdown and boxes
def create_dashboard_elements(data, selected_cluster):
    """
    Function to create the dropdown menu and info boxes for the dashboard.

    Parameters:
    - data: DataFrame containing the cluster data.
    - selected_cluster: The cluster currently selected.

    Returns:
    - A Div containing the dropdown and info boxes.
    """
    # Get the unique cluster values
    cluster_values = sorted(data["Cluster"].dropna().unique())

    # Create the dropdown menu for cluster selection
    dropdown = html.Div(
        [
            html.Label(
                "Seleccionar Cluster:",
                style={
                    "color": "#F4F4F9",       # White label text
                    "fontSize": "16px",       # Font size for better readability
                }
            ),
            dcc.Dropdown(
                id="cluster-dropdown",
                options=[
                    {"label": f"Cluster {int(c)}", "value": int(c)} for c in cluster_values
                ],
                value=selected_cluster,  # Use the selected cluster or default
                placeholder="Seleccione un cluster",  # Placeholder text
                style={
                    "color": "#0C2340",       # Dropdown text color
                    "backgroundColor": "#FFFFFF",  # Dropdown background color
                }
            ),
        ],
        style={"margin-bottom": "20px"}  # Add spacing below the dropdown
    )

    # Create info boxes for displaying additional information
    boxes = html.Div(
        [
            html.Div(
                id="box-femenina",
                style={
                    "border": "1px solid #F4F4F9",
                    "padding": "10px",
                    "margin": "10px",
                    "borderRadius": "5px",
                    "color": "#F4F4F9",
                },
            ),
            html.Div(
                id="box-masculina",
                style={
                    "border": "1px solid #F4F4F9",
                    "padding": "10px",
                    "margin": "10px",
                    "borderRadius": "5px",
                    "color": "#F4F4F9",
                },
            ),
            html.Div(
                id="box-hipotecarios",
                style={
                    "border": "1px solid #F4F4F9",
                    "padding": "10px",
                    "margin": "10px",
                    "borderRadius": "5px",
                    "color": "#F4F4F9",
                },
            ),
            html.Div(
                id="box-automotrices",
                style={
                    "border": "1px solid #F4F4F9",
                    "padding": "10px",
                    "margin": "10px",
                    "borderRadius": "5px",
                    "color": "#F4F4F9",
                },
            ),
            html.Div(
                id="box-personales",
                style={
                    "border": "1px solid #F4F4F9",
                    "padding": "10px",
                    "margin": "10px",
                    "borderRadius": "5px",
                    "color": "#F4F4F9",
                },
            ),
            html.Div(
                id="box-tarjetas",
                style={
                    "border": "1px solid #F4F4F9",
                    "padding": "10px",
                    "margin": "10px",
                    "borderRadius": "5px",
                    "color": "#F4F4F9",
                },
            ),
        ],
        style={
            "display": "grid",
            "gridTemplateColumns": "1fr 1fr",  # Two columns
            "gap": "15px",  # Spacing between boxes
        },
    )

    return html.Div(
        [dropdown, boxes],  # Combine the dropdown and boxes
        style={"padding": "10px"}  # Add some padding to the container
    )

### Applying data to map

In [6]:
# Merge the data with the GeoJSON for Guanajuato
for feature in guanajuato_geojson['features']:
    municipio_name = feature['properties']['NOMGEO']
    cluster_value = data.loc[data['Municipio'] == municipio_name, 'Cluster'].values
    if len(cluster_value) > 0:
        # Convert Cluster value to native Python int
        feature['properties']['Cluster'] = int(cluster_value[0])
    else:
        feature['properties']['Cluster'] = None  # Handle missing values

# Merge the data with the GeoJSON for México
for feature in mexico_geojson['features']:
    municipio_name = feature['properties']['NOMGEO']
    cluster_value = data.loc[data['Municipio'] == municipio_name, 'Cluster'].values
    if len(cluster_value) > 0:
        # Convert Cluster value to native Python int
        feature['properties']['Cluster'] = int(cluster_value[0])
    else:
        feature['properties']['Cluster'] = None  # Handle missing values

# Define the new color palette for clusters
def get_color(cluster):
    color_map = {
        0: 'purple',  # Cluster 0
        1: 'green',   # Cluster 1
        2: 'orange',  # Cluster 2
    }
    return color_map.get(cluster, 'gray')  # Default to gray for missing or unknown clusters

# Create the map
m = folium.Map(location=[19.7, -100.0], zoom_start=7.0)

# Add Guanajuato GeoJSON layer with cluster-based coloring
folium.GeoJson(
    guanajuato_geojson,
    name="Guanajuato",
    style_function=lambda x: {
        'fillColor': get_color(x['properties']['Cluster']),
        'color': 'black',
        'fillOpacity': 0.7,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["NOMGEO", "Cluster"],
        aliases=["Municipio:", "Cluster:"]
    )
).add_to(m)

# Add México GeoJSON layer with cluster-based coloring
folium.GeoJson(
    mexico_geojson,
    name="México",
    style_function=lambda x: {
        'fillColor': get_color(x['properties']['Cluster']),
        'color': 'black',
        'fillOpacity': 0.7,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["NOMGEO", "Cluster"],
        aliases=["Municipio:", "Cluster:"]
    )
).add_to(m)

# Add Layer Control
folium.LayerControl().add_to(m)

# Render the Folium map to an IFrame-compatible HTML string
map_html = m._repr_html_()

### Setting colors per bank

In [7]:
# Color for banks
RAZ_SOCIAL_COLORS = {
    "BANAMEX": "darkblue",
    "BANCO AZTECA": "lightgreen",
    "BANCO DEL BAJIO": "beige",
    "BANCO INBURSA": "lightblue",
    "BANORTE": "red",
    "MULTIVA": "pink",
    "BANCO SANTANDER": "firebrick",
    "BBVA": "blue",
    "BANCOPPEL": "purple",
    "BANJERCITO": "darkgreen",
    "BANCO BANKAOOL": "brown",
    "BANCO HSBC": "orange",
    "BANCO SCOTIABANK": "darkred",
}

### Creating a map with bank merged data

In [8]:
# Function to create the bancos map
def create_bancos_map(data, bancos_data, guanajuato_geojson, mexico_geojson, raz_social_filters):
    m = folium.Map(location=[20.5, -100.5], zoom_start=7)

    cluster_colors = {0: "purple", 1: "green", 2: "orange"}

    # Add GeoJSON layers for Guanajuato
    for feature in guanajuato_geojson.get("features", []):
        municipio_name = feature["properties"]["NOMGEO"]
        cluster_value = data.loc[data["Municipio"] == municipio_name, "Cluster"].values
        color = cluster_colors.get(int(cluster_value[0]), "gray") if len(cluster_value) > 0 else "gray"
        feature["properties"]["Cluster"] = int(cluster_value[0]) if len(cluster_value) > 0 else None

        folium.GeoJson(
            feature,
            style_function=lambda x, color=color: {
                "fillColor": color,
                "color": "black",
                "fillOpacity": 0.7,
            },
            tooltip=folium.GeoJsonTooltip(fields=["NOMGEO", "Cluster"], aliases=["Municipio:", "Cluster:"]),
        ).add_to(m)

    # Add GeoJSON layers for Mexico
    for feature in mexico_geojson.get("features", []):
        municipio_name = feature["properties"]["NOMGEO"]
        cluster_value = data.loc[data["Municipio"] == municipio_name, "Cluster"].values
        color = cluster_colors.get(int(cluster_value[0]), "gray") if len(cluster_value) > 0 else "gray"
        feature["properties"]["Cluster"] = int(cluster_value[0]) if len(cluster_value) > 0 else None

        folium.GeoJson(
            feature,
            style_function=lambda x, color=color: {
                "fillColor": color,
                "color": "black",
                "fillOpacity": 0.7,
            },
            tooltip=folium.GeoJsonTooltip(fields=["NOMGEO", "Cluster"], aliases=["Municipio:", "Cluster:"]),
        ).add_to(m)

    # Filter bancos_data by selected Razón Social if filters are applied
    filtered_bancos = bancos_data[bancos_data["raz_social"].isin(raz_social_filters)] if raz_social_filters else bancos_data[bancos_data["raz_social"] == "BANAMEX"]

    # Add markers for bancos with Razón Social-based colors
    for _, row in filtered_bancos.iterrows():
        color = RAZ_SOCIAL_COLORS.get(row["raz_social"], "gray")
        folium.Marker(
            location=[row["latitud"], row["longitud"]],
            popup=f"Razón Social: {row['raz_social']}",
            icon=folium.Icon(color=color),
        ).add_to(m)

    return m._repr_html_()

### Creating a map with data exclusively for cluster 0

In [9]:
def create_bancos_map_1(cluster_0, bancos_data, guanajuato_geojson, mexico_geojson, raz_social_filters_1):
    m = folium.Map(location=[20.5, -100.5], zoom_start=7)

    cluster_colors = {0: "purple", 1: "green", 2: "orange"}

    # Add GeoJSON layers for Guanajuato
    for feature in guanajuato_geojson.get("features", []):
        municipio_name = feature["properties"]["NOMGEO"]
        # Fetch SubCluster value
        subcluster_value = cluster_0.loc[cluster_0["Municipio"] == municipio_name, "SubCluster"].values
        if len(subcluster_value) == 0 or pd.isnull(subcluster_value[0]):
            # Grey for areas with no SubCluster value
            color = "gray"
            feature["properties"]["SubCluster"] = None  # Explicitly set SubCluster as None
        else:
            # Use cluster color for valid SubCluster values
            color = cluster_colors.get(int(subcluster_value[0]), "gray")
            feature["properties"]["SubCluster"] = int(subcluster_value[0])

        folium.GeoJson(
            feature,
            style_function=lambda x, color=color: {
                "fillColor": color,
                "color": "black",
                "fillOpacity": 0.7,
            },
            tooltip=folium.GeoJsonTooltip(fields=["NOMGEO", "SubCluster"], aliases=["Municipio:", "SubCluster:"]),
        ).add_to(m)

    # Add GeoJSON layers for Mexico
    for feature in mexico_geojson.get("features", []):
        municipio_name = feature["properties"]["NOMGEO"]
        # Fetch SubCluster value
        subcluster_value = cluster_0.loc[cluster_0["Municipio"] == municipio_name, "SubCluster"].values
        if len(subcluster_value) == 0 or pd.isnull(subcluster_value[0]):
            # Grey for areas with no SubCluster value
            color = "gray"
            feature["properties"]["SubCluster"] = None  # Explicitly set SubCluster as None
        else:
            # Use cluster color for valid SubCluster values
            color = cluster_colors.get(int(subcluster_value[0]), "gray")
            feature["properties"]["SubCluster"] = int(subcluster_value[0])

        folium.GeoJson(
            feature,
            style_function=lambda x, color=color: {
                "fillColor": color,
                "color": "black",
                "fillOpacity": 0.7,
            },
            tooltip=folium.GeoJsonTooltip(fields=["NOMGEO", "SubCluster"], aliases=["Municipio:", "SubCluster:"]),
        ).add_to(m)

    # Filter bancos_data by selected Razón Social
    filtered_bancos_1 = bancos_data[bancos_data["raz_social"].isin(raz_social_filters_1)] if raz_social_filters_1 else bancos_data[bancos_data["raz_social"] == "BANAMEX"]

    # Add markers for bancos with Razón Social-based colors only in non-grey areas
    for _, row in filtered_bancos_1.iterrows():
        municipio_name = row["municipio"]  # Use "municipio" for bancos_data
        subcluster_value = cluster_0.loc[cluster_0["Municipio"] == municipio_name, "SubCluster"].values

        # Skip adding markers if the SubCluster value is null (grey areas)
        if len(subcluster_value) == 0 or pd.isnull(subcluster_value[0]):
            continue

        color = RAZ_SOCIAL_COLORS.get(row["raz_social"], "gray")
        folium.Marker(
            location=[row["latitud"], row["longitud"]],
            popup=f"Razón Social: {row['raz_social']}",
            icon=folium.Icon(color=color),
        ).add_to(m)

    return m._repr_html_()

## Creating page Inicio

In [10]:
def page_inicio(pathname, map_html, stored_cluster):
    cluster_values = sorted(data["Cluster"].dropna().unique())
    selected_cluster = stored_cluster if stored_cluster is not None else cluster_values[0]

    if pathname == "/":
        return html.Div(
            [
                # Full-Width Background for Header Area
                html.Div(
                    [
                        html.H1(
                            "BIENVENIDO AL TABLERO BANAMEX",
                            style={
                                "color": "#F4F4F9",  # White text
                                "textAlign": "center",
                                "paddingTop": "40px",  # Add padding to lower the title
                                "paddingBottom": "20px",  # Bottom padding for spacing
                                "fontSize": "40px",
                                "margin": "0",  # Remove default margin
                            }
                        )
                    ],
                    style={
                        "width": "100%",  # Span full page width
                        "backgroundColor": "#0C2340",  # Same color as sidebar
                        "marginBottom": "20px",  # Add space below the header
                    },
                ),
                # Content Area with Expanded Map and Styled Box
                html.Div(
                    [
                        # Expanded Map Area
                        html.Iframe(
                            srcDoc=map_html,
                            style={
                                "width": "70%",  # Increased width
                                "height": "800px",  # Increased height
                                "border": "none",
                                "display": "inline-block",
                            }
                        ),
                        # Dashboard Elements Box Styled to Match Sidebar
                        html.Div(
                            create_dashboard_elements(data, selected_cluster),
                            style={
                                "width": "28%",  # Adjusted width
                                "display": "inline-block",
                                "verticalAlign": "top",
                                "backgroundColor": "#0C2340",  # Dark blue background
                                "padding": "15px",
                                "borderRadius": "10px",
                                "color": "#F4F4F9",  # White text color
                                "boxShadow": "0px 2px 5px rgba(0, 0, 0, 0.1)",  # Subtle shadow
                            }
                        ),
                    ],
                    style={"display": "flex", "justify-content": "space-between"}
                ),
            ],
            style={"padding": "0", "backgroundColor": "#F4F4F9"},  # Remove extra padding
        )

## Creating page Bancos

In [11]:
def page_bancos(raz_social_filters):
    default_filter = ["BANAMEX"] if not raz_social_filters else raz_social_filters

    # Generate the map HTML
    bancos_map_html = create_bancos_map(data, bancos_data, guanajuato_geojson, mexico_geojson, default_filter)

    # Dropdowns for selecting banks
    raz_social_dropdowns = html.Div(
        [
            html.Label(
                "Selecciona hasta 5 bancos:",
                style={"color": "#0C2340", "fontSize": "16px", "fontWeight": "bold"}
            ),
        ]
        + [
            dcc.Dropdown(
                id=f"raz-social-dropdown-{i+1}",
                options=[{"label": raz, "value": raz} for raz in bancos_data["raz_social"].unique()],
                value=default_filter[i] if i < len(default_filter) else None,
                style={
                    "margin-bottom": "10px",
                    "color": "#0C2340",
                    "backgroundColor": "#FFFFFF",
                    "border": "1px solid #D3D3D3",
                },
            )
            for i in range(5)
        ]
        + [
            html.Button(
                "Aplicar",
                id="apply-button",
                n_clicks=0,
                style={
                    "margin-top": "10px",
                    "backgroundColor": "#0072CE",  # Light blue
                    "color": "white",
                    "border": "none",
                    "padding": "10px 20px",
                    "borderRadius": "5px",
                },
            ),
        ],
        style={
            "padding": "15px",
            "border": "1px solid #D3D3D3",
            "backgroundColor": "#FFFFFF",
            "borderRadius": "10px",
            "boxShadow": "0px 2px 5px rgba(0, 0, 0, 0.1)",
        },
    )

    # Selected banks list box
    selected_list_box = html.Div(
        [
            html.H5(
                "Bancos Seleccionados:",
                style={
                    "color": "#000000",  # Black font for the title
                    "fontWeight": "bold",  # Make the title bold
                    "marginBottom": "10px",  # Add spacing below the title
                },
            ),
            html.Ul(
                id="selected-list",
                style={
                    "listStyleType": "none",  # Remove default bullet points
                    "color": "#000000",  # Black font for the list items
                    "padding": "0",  # Remove extra padding
                    "margin": "0",  # Remove extra margins
                },
            ),
        ],
        style={
            "padding": "15px",
            "border": "1px solid #D3D3D3",  # Light gray border
            "backgroundColor": "#FFFFFF",  # White background
            "borderRadius": "10px",  # Rounded corners
            "boxShadow": "0px 2px 5px rgba(0, 0, 0, 0.1)", # Subtle shadow
        },
    )

    # Pie chart (black background)
    pie_chart = dcc.Graph(
        id="pie-chart",
        config={"displayModeBar": False},  # Remove unnecessary toolbar
        style={
            "width": "100%",  # Full width of the container
            "height": "300px",  # Set height to match the pie chart size
            "backgroundColor": "white",  # Black background
        },
    )

    # Treemap
    treemap = dcc.Graph(
        id="treemap",
        config={"displayModeBar": False},  # Remove unnecessary toolbar
        style={
            "width": "100%", 
            "height": "300px",
        },
    )

    # Layout
    return html.Div(
        [
            # Title section
            html.Div(
                html.H1(
                    "BANCO VS BANCO",
                    style={
                        "color": "#F4F4F9",  # White text
                        "textAlign": "center",
                        "paddingTop": "40px",  # Space above the title
                        "paddingBottom": "20px",  # Space below the title
                        "fontSize": "40px",
                        "margin": "0",  # Remove default margins
                    },
                ),
                style={
                    "width": "100%",  # Full width
                    "backgroundColor": "#0C2340",  # Dark blue background
                    "marginBottom": "20px",  # Add spacing below the title section
                },
            ),
            # Map and right-side dropdowns
            html.Div(
                [
                    html.Div(
                        [
                            html.Iframe(
                                srcDoc=bancos_map_html,
                                style={
                                    "width": "100%",
                                    "height": "400px",
                                    "border": "none",
                                },
                            )
                        ],
                        style={"width": "60%",
                               "margin": "0",  # Remove any external margins
                               "padding": "0"},
                    ),
                    html.Div(
                        [
                            raz_social_dropdowns,
                            selected_list_box,
                        ],
                        style={"width": "35%", "margin-left": "10px"},
                    ),
                ],
                style={"display": "flex", "justify-content": "space-between"},
            ),
            # Pie chart and treemap next to each other
            html.Div(
                [
                    html.Div(
                        pie_chart,
                        style={
                            "width": "25%",
                            "margin": "0",  # Remove all margins
                            "padding": "0",  # Remove all padding
                        },
                    ),
                    html.Div(
                        treemap,
                        style={
                            "width": "25%",
                            "margin": "0",  # Remove all margins
                            "padding": "0",  # Remove all padding
                        },
                    ),
                ],
                style={"display": "flex", "margin-top": "20px"},
            ),
        ],
        style={"padding": "0", "backgroundColor": "#F4F4F9"},
    )

## Creating page Comparasion

In [12]:
def page_comparacion():
    return html.Div([
        # Title Section
        html.H1(
            "MUNICIPIO VS MUNICIPIO",
            style={
                "textAlign": "center",
                "margin-bottom": "20px",
                "color": "#0C2340",  # Text color
            },
        ),
        # Cluster Selection Section
        html.Div([
            html.H5(
                "Seleccione Clústeres:",
                style={
                    "textAlign": "center",
                    "margin-bottom": "10px",
                    "color": "#0C2340",  # Text color
                },
            ),
            dcc.Checklist(
                id="cluster-checklist",
                options=[],  # Options will be populated dynamically
                inline=True,
                style={
                    "textAlign": "flex",
                    "justify-content": "center",
                    "margin-bottom": "10px",
                    "gap": "50px",
                    "color": "#0C2340",  # Text color for checklist
                },
            ),
        ], style={"textAlign": "center"}),
        # Municipality Selection Section
        html.Div([
            html.Div([
                dcc.Dropdown(
                    id=f"municipio-dropdown-{i+1}",
                    options=[],  # Options will be populated dynamically
                    placeholder=f"Seleccione Municipio {i+1}",
                    style={
                        "margin-bottom": "30px",
                        "backgroundColor": "white",
                        "color": "#0C2340",  # Text color
                        "width": "100%",
                        "margin": "0 auto",
                    },
                ) for i in range(3)
            ], style={"display": "flex", "justify-content": "center", "margin-bottom": "20px"}),
            html.Div([
                dcc.Dropdown(
                    id=f"municipio-dropdown-{i+4}",
                    options=[],  # Options will be populated dynamically
                    placeholder=f"Seleccione Municipio {i+4}",
                    style={
                        "margin-bottom": "30px",
                        "backgroundColor": "white",
                        "color": "#0C2340",  # Text color
                        "width": "100%",
                        "margin": "0 auto",
                    },
                ) for i in range(2)
            ], style={"display": "flex", "justify-content": "center", "margin-bottom": "20px"}),
        ]),
        # Charts Container
        html.Div(
            id="charts-container",
            style={
                "display": "grid",
                "gridTemplateColumns": "repeat(5, 1fr)",  # 5 charts per row
                "gridGap": "10px",
            },
        ),
    ])

## Creating page SubCluster

In [13]:
def page_subcluster(raz_social_filters_1):
    bancos_map_html = create_bancos_map_1(
        cluster_0, bancos_data, guanajuato_geojson, mexico_geojson, raz_social_filters_1)

    # Bank Selection Dropdown and Apply Button
    raz_social_dropdowns_1 = html.Div(
        [
            html.Label("Select up to 5 banks:", style={"color": "#0C2340", "fontSize": "16px"}),
        ]
        + [
            dcc.Dropdown(
                id=f"raz-social-dropdown-{i+1}",
                options=[{"label": raz, "value": raz} for raz in bancos_data["raz_social"].unique()],
                style={"margin-bottom": "10px", "color": "black"},
            )
            for i in range(5)
        ]
        + [
            html.Button("Apply", id="apply-button", n_clicks=0, style={"margin-top": "10px", "color": "#0C2340"}),
            html.Ul(
                id="selected-list",
                style={"margin-top": "10px", "color": "#0C2340", "listStyleType": "none"},
            ),
        ],
        style={"padding": "10px", "border": "1px solid #ccc", "width": "100%"},
    )

    # Histogram
    histogram = dcc.Graph(id="raz-social-histogram")

    # Detalle por Municipio
    detalle_box = html.Div(
        [
            html.H4("Detalle por Municipio", style={"color": "#0C2340"}),
            dcc.Dropdown(
                id="municipio-dropdown",
                options=[
                    {"label": row["Municipio"], "value": row["Municipio"]}
                    for _, row in cluster_0[cluster_0["Cluster"] == 0].iterrows()
                ],
                style={"margin-bottom": "10px", "color": "black"},
            ),
            html.Div(id="municipio-details", style={"color": "#0C2340"}),
        ],
        style={"padding": "10px", "border": "1px solid #ccc", "width": "100%", "color": "black"},
    )

    return html.Div(
    [
        html.H1(
            "Cluster 0 Maximo Detalle",
            style={
                "textAlign": "center",
                "backgroundColor": "#0C2340",  # Match this color to the sidebar's background
                "color": "white",              # White font color
                "padding": "10px",             # Add some padding for better visibility
                "borderRadius": "5px"          # Optional: Add rounded corners for aesthetics
            }
        ),
        html.Div(
            [
                html.Div(
                    [
                        html.Iframe(
                            srcDoc=bancos_map_html,
                            style={"width": "100%", "height": "400px", "border": "none"},
                        ),
                        histogram,
                    ],
                    style={"width": "60%"},
                ),
                html.Div(
                    [
                        raz_social_dropdowns_1,
                        detalle_box,
                    ],
                    style={"width": "35%", "margin-left": "10px"},
                ),
            ],
            style={"display": "flex", "justify-content": "space-between"},
        ),
    ]
)

# Initializing app and Callbacks

In [14]:
# Create the Dash app with a single call to Dash()
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.DARKLY], suppress_callback_exceptions=True)
app.title = "Tablero Bancos - Consolidated Dashboard"

# Define the new Citibanamex sidebar
sidebar = html.Div(
    [
        html.Div(
            html.Img(
                src="https://h100348845509.wordpress.com/wp-content/uploads/2018/03/banamex-logo1.png",
                alt="Citibanamex logo",
                style={"width": "100%", "padding": "0px", "margin-bottom": "0px"}
            ),
            style={"textAlign": "center"}
        ),
        html.Hr(style={"border-color": "#F4F4F9"}),  # Light gray divider
        dbc.Nav(
            [
                dbc.NavLink("Inicio", href="/", active="exact", style={"color": "#F4F4F9", "font-weight": "bold"}),
                dbc.NavLink("Bancos", href="/bancos", active="exact", style={"color": "#F4F4F9", "font-weight": "bold"}),
                dbc.NavLink("Comparación", href="/comparacion", active="exact", style={"color": "#F4F4F9", "font-weight": "bold"}),
                dbc.NavLink("SubCluster", href="/subcluster", active="exact", style={"color": "#F4F4F9", "font-weight": "bold"}),
            ],
            vertical=True,
            pills=True,
        ),
    ],
    style={
        "position": "fixed",
        "top": 0,
        "left": 0,
        "bottom": 0,
        "width": "18rem",
        "padding": "2rem 1rem",
        "backgroundColor": "#0C2340",  # Dark blue background
        "color": "#F4F4F9",  # Light gray text
    },
)

# Define the layout for the main content area
content = html.Div(
    id="page-content",
    style={
        "margin-left": "18rem",
        "padding": "2rem 1rem",
        "backgroundColor": "#F4F4F9",  # Light gray background for the content
        "min-height": "100vh",  # Ensure full height
    },
)

# Define the app layout
#app.layout = html.Div([dcc.Location(id="url"), sidebar, content])


app.layout = html.Div([
    dcc.Store(id="dropdown-store"),  # Store to preserve dropdown state
    dcc.Store(id="map-store"),      # Store to preserve map state
    dcc.Store(id='selected-cluster-store'),
    dcc.Store(id='raz-social-filters-store', data=[]),  # Filters for Bancos page
    dcc.Store(id='raz-social-filters-1-store', data=[]),  # Filters for Subcluster page
    dcc.Store(id="cluster-store"),
    dcc.Store(id="raz-social-store-0"),
    dcc.Location(id='url'),
    sidebar,
    content,
])

# Callback to render page content based on navigation
@app.callback(
    Output("page-content", "children"),
    [
        Input("url", "pathname"),
        Input("map-store", "data"),
        State("dropdown-store", "data"),
        Input("selected-cluster-store", "data"),
        Input("raz-social-filters-store", "data"),
        Input("raz-social-filters-1-store", "data"),  # Separate store for Subcluster page filters
    ]
)
def render_page_content(pathname, map_data, dropdown_data, selected_cluster, raz_social_filters, raz_social_filters_1):
    if pathname == "/":
        # Use the stored dropdown value or default to the first cluster
        stored_cluster = dropdown_data if dropdown_data is not None else selected_cluster
        return page_inicio(pathname, map_data, stored_cluster)
    elif pathname == "/bancos":
        return page_bancos(raz_social_filters)
    elif pathname == "/comparacion":
        return page_comparacion()
    elif pathname == "/subcluster":
        return page_subcluster(raz_social_filters_1)
    return html.Div("404: Página no encontrada", style={"textAlign": "center"})

# Additional callbacks and function definitions can go here, including those for handling specific data interactions

# Callback for preserving dropdown state
# Callback for preserving dropdown state
@app.callback(
    Output("dropdown-store", "data"),
    [Input("cluster-dropdown", "value")]
)
def update_dropdown_store(selected_cluster):
    return selected_cluster


# Callback for updating the map
@app.callback(
    Output("map-store", "data"),
    [Input("cluster-dropdown", "value")]
)
def update_map(selected_cluster):
    # Update the map based on the selected cluster
    return create_map(data, guanajuato_geojson, mexico_geojson, selected_cluster)

# Callback for updating the boxes
@app.callback(
    [
        Output("box-femenina", "children"),
        Output("box-masculina", "children"),
        Output("box-hipotecarios", "children"),
        Output("box-automotrices", "children"),
        Output("box-personales", "children"),
        Output("box-tarjetas", "children"),
    ],
    [Input("cluster-dropdown", "value")]
)
def update_boxes(cluster):
    # Filter for the specific cluster
    filtered_data = data[data["Cluster"] == cluster]
    med_femenina = f"Población Femenina (Mediana): {filtered_data['Poblacion_adulta_mujeres'].median()}"
    med_masculina = f"Población Masculina (Mediana): {filtered_data['Poblacion_adulta_hombres'].median()}"
    med_hipotecarios = f"Número de Créditos Hipotecarios (Mediana): {filtered_data['Hipotecario_numero_creditos'].median()}"
    med_automotrices = f"Número de Créditos Automotrices (Mediana): {filtered_data['Automotriz_numero_creditos'].median()}"
    med_personales = f"Número de Créditos Personales (Mediana): {filtered_data['Personal_numero_creditos'].median()}"
    med_tarjetas = f"Número de Tarjetas de Crédito (Mediana): {filtered_data['Tarjetas_credito_numero'].median()}"

    return [med_femenina, med_masculina, med_hipotecarios, med_automotrices, med_personales, med_tarjetas]



@app.callback(
    Output("raz-social-filters-store", "data"),  # Store for applied filters
    [Input("apply-button", "n_clicks")],  # Trigger on Apply button
    [State(f"raz-social-dropdown-{i+1}", "value") for i in range(5)]  # Get dropdown values
)
def update_filters(n_clicks, *filters):
    # Remove None values and ensure at least "BANAMEX" is included if no selections
    applied_filters = [f for f in filters if f] or ["BANAMEX"]
    return applied_filters

@app.callback(
    [
        Output("pie-chart", "figure"),  # Update pie chart
        Output("selected-list", "children"),  # Update list of selected banks
    ],
    [Input("raz-social-filters-store", "data")],  # Trigger on applied filters
)
def update_pie_chart_and_list(raz_social_filters):
    # Handle empty filters (default to BANAMEX)
    if not raz_social_filters:
        raz_social_filters = ["BANAMEX"]

    # Filter data based on selected banks
    filtered_data = bancos_data[bancos_data["raz_social"].isin(raz_social_filters)]
    total_count = bancos_data.shape[0]

    # Pie chart data
    pie_data = filtered_data["raz_social"].value_counts()
    pie_data_percent = (pie_data / total_count * 100).reset_index()
    pie_data_percent.columns = ["raz_social", "percentage"]

    # Create pie chart without displaying labels
    pie_fig = px.pie(
        pie_data_percent,
        values="percentage",
        title="Distribución de Bancos",
        color="raz_social",
        color_discrete_map=RAZ_SOCIAL_COLORS,
    )
    pie_fig.update_traces(
        textinfo="percent",  # Only display percentages on the chart
        hoverinfo="label",  # Show labels and percentages in hover text
        textfont_size=20,  # Increase font size for better readability
    )

    # Adjust the figure layout to make the chart bigger
    pie_fig.update_layout(
        height=300,  # Increase chart height
        width=400,   # Increase chart width
        showlegend=False,  # Hide legend as it is redundant without labels
        title={
            "text": "Distribución de Bancos",
            "x": 0.5,  # Center the title
            "xanchor": "center",
        },
    )

    # List with color-coded squares
    selected_list_items = [
        html.Li(
            [
                html.Span(
                    style={
                        "backgroundColor": RAZ_SOCIAL_COLORS.get(raz, "gray"),
                        "display": "inline-block",
                        "width": "20px",
                        "height": "20px",
                        "marginRight": "10px",
                    }
                ),
                f"{raz} ({pie_data[raz]} sucursales)" if raz in pie_data.index else f"{raz} (0 sucursales)",
            ],
            style={"margin-bottom": "10px"},
        )
        for raz in raz_social_filters
    ]

    return pie_fig, selected_list_items

@app.callback(
    Output("treemap", "figure"),  # Update treemap
    [Input("raz-social-filters-store", "data")]  # Trigger on applied filters
)
def update_treemap(raz_social_filters):
    # Handle empty filters (default to BANAMEX)
    if not raz_social_filters:
        raz_social_filters = ["BANAMEX"]

    # Filter data based on selected banks
    filtered_data = bancos_data[bancos_data["raz_social"].isin(raz_social_filters)]

    # Count of municipalities with presence for each selected bank
    treemap_data = (
        filtered_data.groupby("raz_social")["municipio"]
        .nunique()
        .reset_index()
        .rename(columns={"municipio": "count"})
    )

    # Create treemap
    treemap_fig = px.treemap(
        treemap_data,
        path=["raz_social"],
        values="count",
        color="raz_social",
        color_discrete_map=RAZ_SOCIAL_COLORS,
    )

    # Update hover information to show only 'label' and 'count'
    treemap_fig.update_traces(
        hovertemplate="<b>%{label}</b><br>Count: %{value}<extra></extra>"
    )

    # Adjust the layout
    treemap_fig.update_layout(
        title="Cantidad de Municipios<br>Con Presencia del Banco",
        title_x=0.5,  # Center the title
        margin=dict(t=50, l=10, r=10, b=10),  # Reduce margins
    )

    return treemap_fig

# Load data dynamically
@app.callback(
    Output("cluster-checklist", "options"),
    Input("url", "pathname")
)
def load_cluster_options(pathname):
    if pathname == "/comparacion":
        # Assuming `data` is a DataFrame with a 'Cluster' column
        clusters = data["Cluster"].unique()
        return [{"label": f"Cluster {cluster}", "value": cluster} for cluster in clusters]
    return []

# Update dropdown options based on selected clusters
@app.callback(
    [Output(f"municipio-dropdown-{i+1}", "options") for i in range(5)],
    Input("cluster-checklist", "value")
)
def update_municipios(selected_clusters):
    if not selected_clusters:
        return [[] for _ in range(5)]
    filtered_data = data[data["Cluster"].isin(selected_clusters)]
    municipios = filtered_data["Municipio"].unique()
    options = [{"label": municipio, "value": municipio} for municipio in municipios]
    return [options for _ in range(5)]

# Generate bar charts based on selected municipios
@app.callback(
    Output("charts-container", "children"),
    [Input(f"municipio-dropdown-{i+1}", "value") for i in range(5)]
)
def generate_charts(*selected_municipios):
    selected_municipios = [m for m in selected_municipios if m]
    if not selected_municipios:
        return []
    charts = []
    variables = [
        ("Poblacion_adulta_mujeres", "Poblacion de<br>Mujeres"),
        ("Tarjetas_credito_numero", "Poblacion de<br>Hombres"),
        ("Personal_numero_creditos", "Numero de<br>Creditos Personales"),
        ("Hipotecario_numero_creditos", "Numero de<br>Creditos Hipotecarios"),
        ("Automotriz_numero_creditos", "Numero de<br>Creditos Automotriz"),
        ("Viviendas_particulares_habitadas_que_disponen_de_consola_de_videojuegos", "Viviendas con<br>Consola de Videojuegos"),
        ("Viviendas_particulares_habitadas_que_disponen_de_servicio_de_peliculas_musica_o_videos_de_paga_por_internet", "Viviendas con<br>Servicio de Streaming"),
        ("Viviendas_particulares_habitadas_que_disponen_de_internet", "Viviendas que<br>Disponen de Internet"),
        ("Viviendas_particulares_habitadas_que_disponen_de_servicio_de_television_de_paga", "Viviendas que<br>Disponen de<br>Television Pagada"),
        ("Viviendas_particulares_habitadas_que_disponen_de_linea_telefonica_fija", "Viviendas que<br>Disponen de<br>Linea Telefonica")
    ]

    for var, label in variables:
        filtered_data = data[data["Municipio"].isin(selected_municipios)]
        fig = px.bar(filtered_data, x="Municipio", y=var, title=label)
        fig.update_layout(
            title={"text": label, "x": 0.5, "xanchor": "center", "y": 0.95, "yanchor": "top"},
            margin=dict(l=20, r=20, t=80, b=20),  # Adjusted margins to provide more space for a wrapped title
            yaxis_title=None,
            xaxis_title="",  # Remove the label "Municipio"
            title_font={"size": 16, "family": "Arial", "color": "darkblue"}  # Adjust font properties as needed
        )
        charts.append(dcc.Graph(figure=fig))
    return charts

# Placeholder: Additional callbacks to handle histogram and municipio details
@app.callback(
    Output("municipio-details", "children"),
    [Input("municipio-dropdown", "value")],
)
def update_municipio_details(selected_municipio_1):
    if not selected_municipio_1:
        return html.P("Seleccione un municipio para ver los detalles.", style={"color": "gray"})

    # Filter the data for the selected municipio
    municipio_data = cluster_0[cluster_0["Municipio"] == selected_municipio_1]
    if municipio_data.empty:
        return html.P("No hay datos disponibles para este municipio.", style={"color": "gray"})

    municipio_data = municipio_data.iloc[0]  # Get the first (and only) row

    # Create mini boxes with details
    details = [
        html.Div(
            [
                html.Div(
                    f"Población Hombres: {municipio_data['Poblacion_adulta_hombres']}",
                    style={"padding": "5px", "border": "1px solid #0C2340", "borderRadius": "5px"},
                ),
                html.Div(
                    f"Población Mujeres: {municipio_data['Poblacion_adulta_mujeres']}",
                    style={"padding": "5px", "border": "1px solid #0C2340", "borderRadius": "5px"},
                ),
                html.Div(
                    f"Tarjetas de Crédito: {municipio_data['Tarjetas_credito_numero']}",
                    style={"padding": "5px", "border": "1px solid #0C2340", "borderRadius": "5px"},
                ),
                html.Div(
                    f"Número de Créditos Personales: {municipio_data['Personal_numero_creditos']}",
                    style={"padding": "5px", "border": "1px solid #0C2340", "borderRadius": "5px"},
                ),
                html.Div(
                    f"Número de Créditos Hipotecarios: {municipio_data['Hipotecario_numero_creditos']}",
                    style={"padding": "5px", "border": "1px solid #0C2340", "borderRadius": "5px"},
                ),
                html.Div(
                    f"Número de Créditos Automotrices: {municipio_data['Automotriz_numero_creditos']}",
                    style={"padding": "5px", "border": "1px solid #0C2340", "borderRadius": "5px"},
                ),
                html.Div(
                    f"Viviendas con Teléfonos Celulares: {municipio_data['Viviendas_particulares_habitadas_que_disponen_de_telefono_celular']}",
                    style={"padding": "5px", "border": "1px solid #0C2340", "borderRadius": "5px"},
                ),
                html.Div(
                    f"Viviendas con Internet: {municipio_data['Viviendas_particulares_habitadas_que_disponen_de_internet']}",
                    style={"padding": "5px", "border": "1px solid #0C2340", "borderRadius": "5px"},
                ),
                html.Div(
                    f"Viviendas con Servicios de Streaming: {municipio_data['Viviendas_particulares_habitadas_que_disponen_de_servicio_de_peliculas_musica_o_videos_de_paga_por_internet']}",
                    style={"padding": "5px", "border": "1px solid #0C2340", "borderRadius": "5px"},
                ),
                html.Div(
                    f"Viviendas con Consola de Videojuegos: {municipio_data['Viviendas_particulares_habitadas_que_disponen_de_consola_de_videojuegos']}",
                    style={"padding": "5px", "border": "1px solid #0C2340", "borderRadius": "5px"},
                ),
            ],
            style={"display": "grid", "gridTemplateColumns": "1fr 1fr", "gap": "10px"},
        )
    ]

    return details

@app.callback(
    Output("raz-social-filters-1-store", "data"),
    Input("apply-button", "n_clicks"),  # Triggered by the Apply button
    [State(f"raz-social-dropdown-{i+1}", "value") for i in range(5)]  # States from dropdowns
)
def update_filters_subcluster(n_clicks, *filters):
    if n_clicks is None:  # Handle the case where the button hasn't been clicked
        return []
    # Flatten and remove None values from filters
    return [f for f in filters if f]

@app.callback(
    Output("bancos-map", "srcDoc"),
    [Input("raz-social-1-store", "data")],
)
def update_map_1(raz_social_filters_1):
    bancos_map_html = create_bancos_map_1(cluster_0, bancos_data, guanajuato_geojson, mexico_geojson, raz_social_filters_1)
    return bancos_map_html

from dash import callback_context  # Import this for handling multiple triggers

@app.callback(
    Output("raz-social-histogram", "figure"),
    [
        Input("raz-social-filters-1-store", "data"),  # Applied bank filters
    ]
)
def update_histogram(raz_social_filters_1):
    # Ensure `raz_social_filters_1` is a list and not empty
    if not raz_social_filters_1:
        raz_social_filters_1 = bancos_data["raz_social"].unique()  # Default to all banks
        color_map = {raz: "darkblue" for raz in raz_social_filters_1}  # Default to a single color
    else:
        # Use predefined colors for selected filters
        color_map = {raz: RAZ_SOCIAL_COLORS.get(raz, "gray") for raz in raz_social_filters_1}

    # Exclude municipios from grey areas in the map (Cluster/SubCluster is None)
    excluded_municipios = []
    
    # Check Guanajuato GeoJSON for grey areas
    for feature in guanajuato_geojson["features"]:
        if feature["properties"].get("SubCluster") is None:  # Highlighted as grey
            excluded_municipios.append(feature["properties"]["NOMGEO"])
    
    # Check Mexico GeoJSON for grey areas
    for feature in mexico_geojson["features"]:
        if feature["properties"].get("SubCluster") is None:  # Highlighted as grey
            excluded_municipios.append(feature["properties"]["NOMGEO"])

    # Filter bancos_data to exclude grey areas
    filtered_bancos = bancos_data[
        (bancos_data["raz_social"].isin(raz_social_filters_1)) &
        (~bancos_data["municipio"].isin(excluded_municipios))
    ]

    # Create the histogram
    fig = px.histogram(
        filtered_bancos,
        x="raz_social",
        title="Presencia Física por Banco",
        color="raz_social",
        color_discrete_map=color_map,
    )

    # Update figure layout
    fig.update_layout(
        xaxis_title="Banco",
        yaxis_title="Cantidad",
        showlegend=False,
        plot_bgcolor="white",
        paper_bgcolor="white",
        font_color="darkblue",
        yaxis=dict(
        gridcolor="lightblue"  # Light blue horizontal grid lines
        ),
    )


    return fig

if __name__ == "__main__":
    app.run_server(port=8119, debug=True)

Dash app running on http://127.0.0.1:8119/


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/dash/dash.py:579: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.

